In [1]:
import dlib
import cv2
from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance
import json

In [2]:
def convert_to_ela_image(path, quality, label):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.png'
    ELA_filename = filename.split('/')[-1].split('.')[0] + '.ela.png'
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'PNG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    ela_im = ImageChops.difference(im, resaved_im)
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    ela_im.save('train/'+label+'/'+ELA_filename)

In [3]:
train_frame_folder = 'train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)
list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]
detector = dlib.get_frontal_face_detector()
for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('dataset/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('dataset/fake/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                count+=1

In [4]:
#convert to ela image
# real_frame_folder = 'dataset/real'
# fake_frame_folder = 'dataset/fake'
# list_of_real_data = [f for f in os.listdir(real_frame_folder) if f.endswith('.png')]
# list_of_fake_data = [f for f in os.listdir(fake_frame_folder) if f.endswith('.png')]
# for rd in list_of_real_data:
#     convert_to_ela_image(real_frame_folder+'/'+rd, 90, 'real')
# for fd in list_of_fake_data:
#     convert_to_ela_image(fake_frame_folder+'/'+fd, 90, 'fake')